# Lab 3.4
## Información general
Este laboratorio es una continuación de los laboratorios guiados del Módulo 3.

En este laboratorio, dividirá los datos en tres conjuntos de datos separados:

Conjunto de entrenamiento: se utilizará para entrenar el modelo.
Conjunto de validación: se utilizará durante el entrenamiento para validar el modelo.
Conjunto de prueba: este se retendrá y se utilizará para producir métricas después de entrenar el modelo. Utilizará este conjunto de datos en un próximo laboratorio.
Con los datos divididos, entrenará un modelo XGBoost.

In [ ]:
import warnings, requests, zipfile, io
warnings.simplefilter('ignore')
import pandas as pd
from scipy.io import arff

In [ ]:
# Descarga el archivo zip
f_zip = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'
r = requests.get(f_zip, stream=True)
Vertebral_zip = zipfile.ZipFile(io.BytesIO(r.content))

#Extracción del contenido
Vertebral_zip.extractall()

In [ ]:
# Lectura del archivo arff
data = arff.loadarff('column_2C_weka.arff')

# Creación del DataFrame con Pandas
df = pd.DataFrame(data[0])
df.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,b'Abnormal'
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,b'Abnormal'
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,b'Abnormal'
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,b'Abnormal'
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,b'Abnormal'


In [ ]:
# Codificación de variables categóricas
class_mapper = {b'Abnormal':1,b'Normal':0}
df['class']=df['class'].replace(class_mapper)

# Paso 1: Explorar los datos

In [ ]:
# Visualización de las dimensiones del DataFrame
df.shape

(310, 7)

In [ ]:
# Visualización de las columnas del DataFrame
df.columns

Index(['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle',
       'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis', 'class'],
      dtype='object')

# Paso 2: Preparar los datos

In [ ]:
# Trasladar la columna objetivo al principio
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

In [ ]:
df.columns

Index(['class', 'pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle',
       'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis'],
      dtype='object')

## Separación de datos de entrenamiento, validación y pruebas

In [ ]:
from sklearn.model_selection import train_test_split
train, test_and_validate = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])

In [ ]:
test, validate = train_test_split(test_and_validate, test_size=0.5, random_state=42, stratify=test_and_validate['class'])

In [ ]:
# Visualización de las dimensiones de los dataFrame
print(train.shape)
print(test.shape)
print(validate.shape)

(248, 7)
(31, 7)
(31, 7)


In [ ]:
# Visualización de las proporciones
print(train['class'].value_counts())
print(test['class'].value_counts())
print(validate['class'].value_counts())

class
1    168
0     80
Name: count, dtype: int64
class
1    21
0    10
Name: count, dtype: int64
class
1    21
0    10
Name: count, dtype: int64


## Entrenamiento del modelo XGBoost
El modelo XGBoost (eXtreme Gradient Boosting) es un algoritmo de aprendizaje automático de código abierto y muy popular, que combina árboles de decisión con la técnica de boosting (potenciación de gradiente) para crear modelos predictivos potentes y precisos, destacando por su velocidad, eficiencia, paralelización, escalabilidad en grandes datos

In [ ]:
train.columns

Index(['class', 'pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle',
       'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis'],
      dtype='object')

In [ ]:
# Separar X e y para cada conjunto
x_train = train.drop(columns=['class'])
y_train = train['class']

x_test = test.drop(columns=['class'])
y_test = test['class']

x_val = validate.drop(columns=['class'])
y_val = validate['class']

In [ ]:
# Instalar el modelo XGBoost
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:

# Modelo XGBoost optimizado para Colab
xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    eval_metric='logloss',
    tree_method='hist',
    device='cuda'
)

# Entrenamiento con evaluación en validación

xgb_model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=50
)

[0]	validation_0-logloss:0.61441
[50]	validation_0-logloss:0.29701
[100]	validation_0-logloss:0.24016
[150]	validation_0-logloss:0.23405
[200]	validation_0-logloss:0.22862
[250]	validation_0-logloss:0.21807
[299]	validation_0-logloss:0.21368


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

### Evaluación del modelo

---

#### Exactitud (Accuracy)

Proporción de predicciones correctas sobre el total de predicciones.
Indica **qué tan a menudo el modelo acierta**.

---

#### Precisión (Precision)

De todos los casos que el modelo predijo como **positivos**, ¿cuántos realmente lo eran?
Mide **qué tan confiable es el modelo cuando dice que algo es positivo**.

---

#### Recall (Sensibilidad)

De todos los casos que **sí eran positivos**, ¿cuántos detectó el modelo?
Mide **qué tanto se le escapan los positivos reales**.

---

#### F1-Score

Promedio armonizado entre **precisión y recall**.

---


In [ ]:
# Evaluación del modelo
y_pred = xgb_model.predict(x_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Accuracy: 0.8064516129032258

Classification Report:

              precision    recall  f1-score   support

           0       0.75      0.60      0.67        10
           1       0.83      0.90      0.86        21

    accuracy                           0.81        31
   macro avg       0.79      0.75      0.77        31
weighted avg       0.80      0.81      0.80        31

